In [ ]:
import tweepy
import json
from pymongo import MongoClient
from bson.objectid import ObjectId
from bson.json_util import dumps
import time
import math

In [ ]:
CONSUMER_KEY = ''
CONSUMER_SECRET = ''
ACCESS_TOKEN = ''
ACCESS_TOKEN_SECRET = ''

In [ ]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:
MONGO_HOST = 'mongodb://localhost:27017/'

client = MongoClient(MONGO_HOST)

db = client['8M2020_sample']

collection = db.tweets_sample

collection2 = db.tweets_sample_updated

In [ ]:
num_docs = collection.count_documents({})
num_limit = 1000
num_skip = 900
# num_docs

In [ ]:
num_page = 0
pages = math.ceil(num_docs/num_limit)
# pages

In [ ]:
print('Atualização de tweets iniciada')
print(f'{time.asctime(time.localtime(time.time()))}')

while num_page < pages:
    print(f'\nBloco {num_page + 1}/{pages}')

    docs = []
    tweets_ids = []

    docs = list(collection.find({}, {'id': True}
                                ).limit(num_limit).skip(num_skip))
    tweets_ids = [doc['id'] for doc in docs]

    print(f'Extraindo {len(tweets_ids)} tweets para atualização\n')

    data_to_update = {}
    data = []
    key = 0

    for tweet_id in tweets_ids:
        data.append(tweet_id)
        data_to_update[key] = data
        if(len(data) == 100):
            key = key + 1
            data = []

    for d in data_to_update.keys():
        updated_tweets = []
        tweets = []

        updated_tweets = api.statuses_lookup(
            data_to_update[d], tweet_mode='extended')
        tweets = [ut._json for ut in updated_tweets]
        collection2.insert_many(tweets)

        num_skip = num_skip + len(data_to_update[d])

        print(
            f'{len(data_to_update[d])} tweets enviados | {len(tweets)} atualizados | Total enviado: {num_skip}/{num_docs} | {(num_skip*100)/num_docs:.2f}%')

    num_page = num_page + 1

print('Atualização de tweets finalizada')
print(time.asctime(time.localtime(time.time())))

In [ ]:
# collection2.drop()